<a href="https://colab.research.google.com/github/KyklyMachine/NLP_SVM/blob/main/First_SVM_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import sklearn
import pandas as pd
from google.colab import files
from nltk.tokenize import word_tokenize
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score


In [49]:
files.upload()

Saving Imported-table-Grid-view-3.csv to Imported-table-Grid-view-3 (1).csv


{'Imported-table-Grid-view-3.csv': b'\xef\xbb\xbfid,rss_id,user_id,res_score,rss_text\n1,fjwenjwef,2,1,dawa awdsa. dawawd dsa\n2,wjbejhwbh,3,2,fer erg ef l jj jo jj \n3,wfekwnf,1,3,nf jknkjfnr Jan nn krnf kj\n4,efjwfj,3,4,f ka kjj jwjke fake k\n5,3u4bf,2,5, ervjkenjen jk k ekn kwnk\n6,ffwjfjs,1,5, ewjkefnn n Jan. ow q\n7,fmwnefj,2,4, rkjnj. n nine mklnew\n8,ehwjcbjds,2,3, woken nkj n n wek m \n9,cwjbjds,1,2,ewkjn kine wn kjnwe\n10,cewjhcb,4223,1,audio n wnjkanjndaw nj bawbdj ba ba bhdb\n,,,,\n,,,,\n,,,,\n,,,,\n,,,,'}

In [50]:
data = pd.read_csv("Imported-table-Grid-view-3.csv")

In [51]:
data.head(10)

,id,rss_id,user_id,res_score,rss_text
0,1.0,fjwenjwef,2.0,1.0,dawa awdsa. dawawd dsa
1,2.0,wjbejhwbh,3.0,2.0,fer erg ef l jj jo jj
2,3.0,wfekwnf,1.0,3.0,nf jknkjfnr Jan nn krnf kj
3,4.0,efjwfj,3.0,4.0,f ka kjj jwjke fake k
4,5.0,3u4bf,2.0,5.0,ervjkenjen jk k ekn kwnk
5,6.0,ffwjfjs,1.0,5.0,ewjkefnn n Jan. ow q
6,7.0,fmwnefj,2.0,4.0,rkjnj. n nine mklnew
7,8.0,ehwjcbjds,2.0,3.0,woken nkj n n wek m
8,9.0,cwjbjds,1.0,2.0,ewkjn kine wn kjnwe
9,10.0,cewjhcb,4223.0,1.0,audio n wnjkanjndaw nj bawbdj ba ba bhdb


In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   res_score  10 non-null     float64
 1   rss_text   10 non-null     object 
dtypes: float64(1), object(1)
memory usage: 368.0+ bytes


In [53]:
data.isnull().sum()

id           5
rss_id       5
user_id      5
res_score    5
rss_text     5
dtype: int64

In [56]:
data = data.drop(["id", "rss_id", "user_id"], axis=1)

In [79]:
data.rss_text.astype(str)
data.head(10)

,res_score,rss_text
0,1.0,dawa awdsa. dawawd dsa
1,2.0,fer erg ef l jj jo jj
2,3.0,nf jknkjfnr Jan nn krnf kj
3,4.0,f ka kjj jwjke fake k
4,5.0,ervjkenjen jk k ekn kwnk
5,5.0,ewjkefnn n Jan. ow q
6,4.0,rkjnj. n nine mklnew
7,3.0,woken nkj n n wek m
8,2.0,ewkjn kine wn kjnwe
9,1.0,audio n wnjkanjndaw nj bawbdj ba ba bhdb


In [102]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [83]:
data["tokenized_rss_text"] = data["rss_text"].fillna("").map(nltk.word_tokenize)
data.head(10)

,res_score,rss_text,tokenized_rss_text
0,1.0,dawa awdsa. dawawd dsa,"[dawa, awdsa, ., dawawd, dsa]"
1,2.0,fer erg ef l jj jo jj,"[fer, erg, ef, l, jj, jo, jj]"
2,3.0,nf jknkjfnr Jan nn krnf kj,"[nf, jknkjfnr, Jan, nn, krnf, kj]"
3,4.0,f ka kjj jwjke fake k,"[f, ka, kjj, jwjke, fake, k]"
4,5.0,ervjkenjen jk k ekn kwnk,"[ervjkenjen, jk, k, ekn, kwnk]"
5,5.0,ewjkefnn n Jan. ow q,"[ewjkefnn, n, Jan., ow, q]"
6,4.0,rkjnj. n nine mklnew,"[rkjnj, ., n, nine, mklnew]"
7,3.0,woken nkj n n wek m,"[woken, nkj, n, n, wek, m]"
8,2.0,ewkjn kine wn kjnwe,"[ewkjn, kine, wn, kjnwe]"
9,1.0,audio n wnjkanjndaw nj bawbdj ba ba bhdb,"[audio, n, wnjkanjndaw, nj, bawbdj, ba, ba, bhdb]"


In [103]:
# Step - a : Remove blank rows if any.
###data["rss_text"].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
###data["tokenized_rss_text"] = [entry.lower() for entry in data["tokenized_rss_text"]]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(data["tokenized_rss_text"]):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    data.loc[index,'text_final'] = str(Final_words)
    #Далее везде заменяем rss_text на text_final

In [104]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['text_final'],data['res_score'],test_size=0.3)

In [105]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [107]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [112]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  40.0
